In [ ]:
import pickle

import modules.engineer as eng
import modules.features as feat
import modules.secretary as secr
import modules.translator as trans
from importlib import reload
reload(eng)
reload(feat)
reload(secr)
reload(trans)
print()

In [ ]:
# Real-Sociedad
list(trans.fbref_com_links().items())[-7:-6]

In [ ]:
from steps.step_01 import DataScraper

teams = trans.fbref_com_links().items()
no_of_seasons = 1
no_of_recursions = 15

step_01 = DataScraper(teams, no_of_seasons, no_of_recursions)
step_01.do()

In [ ]:
from steps.step_02 import DataMonitor

step_02 = DataMonitor()
step_02.do()

In [ ]:
from steps.step_03 import ProductionMaker

starting_season = 2017

step_03 = ProductionMaker(starting_season)
step_03.do()

In [ ]:
from steps.step_04 import ProductionMonitor

step_04 = ProductionMonitor()
step_04.do()

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.feature_selection import RFECV, SelectFromModel
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import (RBF,
                                              DotProduct,
                                              Matern,
                                              RationalQuadratic,
                                              WhiteKernel)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import (KBinsDiscretizer,
                                   MinMaxScaler,
                                   StandardScaler)
from sklearn.svm import SVC

from steps.step_05 import ModelMaker 

sfm = SelectFromModel(RandomForestClassifier(n_estimators=10, random_state=42))
cv = 3

lrc = {'name': 'LogisticRegression',
       'classifier': CalibratedClassifierCV(base_estimator=LogisticRegression(random_state=42,
                                                                              max_iter=100000), cv=cv),
       'param_grid': [{'preprocessor__num__scaling': [None],
                       'selector': [sfm],
                       'selector__threshold': [f"{x/10}*median" for x in range(1, 6, 2)],
                       'classifier__base_estimator__C': [10**-x for x in range(0, 7, 2)],
                       'classifier__base_estimator__class_weight': [None],
                       }]}

rfc = {'name': 'RandomForestClassifier',
       'classifier': CalibratedClassifierCV(base_estimator=RandomForestClassifier(random_state=42), cv=cv),
       'param_grid': [{'preprocessor__num__binning': [None],
                       'selector': [sfm],
                       'selector__threshold': [f"{x/10}*median" for x in range(5, 20, 7)],
                       'classifier__base_estimator__n_estimators': [x for x in range(50, 201, 75)],
                       'classifier__base_estimator__max_depth': [x for x in range(15, 86, 35)],
                       'classifier__base_estimator__min_samples_leaf': [x for x in range(5, 36, 15)],
                       'classifier__base_estimator__class_weight': [None],
                       }]}

mlp = {'name': 'MLPClassifier',
       'classifier': CalibratedClassifierCV(base_estimator=MLPClassifier(random_state=42,
                                                                         max_iter=10000), cv=cv),
       'param_grid': [{'preprocessor__num__scaling': [None],
                       'selector': [sfm],
                       'selector__threshold': [f"{x/10}*median" for x in range(5, 20, 7)],
                       'classifier__base_estimator__hidden_layer_sizes': [[10], [10, 10], [10, 10, 10]],
                       'classifier__base_estimator__alpha': [1*10**-x for x in range(-1, 4, 2)],
                       }]}

knc = {'name': 'KNeighborsClassifier',
       'classifier': CalibratedClassifierCV(base_estimator=KNeighborsClassifier(), cv=cv),
       'param_grid': [{'preprocessor__num__scaling': [StandardScaler(), None],
                       'preprocessor__num__binning': [None],
                       'preprocessor__num__poly': [None],
                       'selector': [sfm],
                       'selector__threshold': [f"{x/10}*median" for x in range(1, 4, 1)],
                       'classifier__base_estimator__n_neighbors': [200, 250, 300]
                       }]}

abc = {'name': 'AdaBoostClassifier',
       'classifier': CalibratedClassifierCV(base_estimator=AdaBoostClassifier(), cv=cv),
       'param_grid': [{'preprocessor__num__scaling': [StandardScaler(), None],
                       'preprocessor__num__binning': [None],
                       'preprocessor__num__poly': [None],
                       'selector': [sfm],
                       'selector__threshold': [f"{x/1000}*median" for x in range(25, 76, 25)],
                       'classifier__base_estimator__n_estimators': [x for x in range(5, 15, 5)],
                       'classifier__base_estimator__learning_rate': [x/10 for x in range(16, 21, 1)]
                       }]}

models = []
models.append(lrc)
models.append(rfc)
models.append(mlp)
models.append(knc)
models.append(abc)

step_05 = ModelMaker(secr.load_model(),
                     feat.num_features(),
                     feat.cat_features(),
                     feat.label(),
                     models,
                     False)
step_05.do()

In [ ]:
from steps.step_06 import GamesScraper

matchweeks = {"Bundesliga": "Spielwoche 33",
              "Premier League": "Spielwoche 36",
              "La Liga": "Spielwoche 36",
              "Serie A": "Spielwoche 36",
              "Ligue 1": "Spielwoche 36"}

step_06 = GamesScraper(matchweeks, "2021-2022", 2022)
step_06.do()

In [ ]:
from steps.step_07 import PredictionMaker

current_season = "2021-2022"

step_07 = PredictionMaker(current_season)
step_07.do()

In [ ]:
from steps.step_08 import PredictionMonitor

step_08 = PredictionMonitor()
step_08.do()

In [ ]:
import pickle
from steps.step_09 import PredictionMaker

lrc = pickle.load(open('./sources/models/LogisticRegression.sav', 'rb'))
rfc = pickle.load(open('./sources/models/RandomForestClassifier.sav', 'rb'))
mlp = pickle.load(open('./sources/models/MLPClassifier.sav', 'rb'))
knc = pickle.load(open('./sources/models/KNeighborsClassifier.sav', 'rb'))
abc = pickle.load(open('./sources/models/AdaBoostClassifier.sav', 'rb'))

models = {'lrc': lrc,
          'rfc': rfc,
          'mlp': mlp,
          'knc': knc,
          'abc': abc}

step_09 = PredictionMaker(models)
step_09.do()